In [0]:
CREATE OR REPLACE TEMPORARY FUNCTION clean_date(d DATE)
RETURNS DATE
RETURN CASE
  WHEN d IS NULL THEN NULL
  WHEN d < DATE '1901-01-01' THEN NULL
  WHEN d IN (
    DATE '1900-01-01', DATE '1901-01-01',
    DATE '1969-12-31', DATE '1970-01-01',
    DATE '2099-12-31', DATE '2999-12-31', DATE '9999-12-31'
  ) THEN NULL
  WHEN d > current_date() + INTERVAL 3 YEARS THEN NULL
  ELSE d
END;

--CREATE OR REPLACE TABLE teamconnect.pca_prism.01_tbl_account_prism_payments AS
INSERT OVERWRITE TABLE pca_prism.01_tbl_account_prism_payments
WITH pay_raw AS (
  SELECT
    REGEXP_REPLACE(CAST(paypex AS STRING),'[^0-9]','') AS exc_digits,
    REGEXP_REPLACE(CAST(paypln AS STRING),'[^0-9]','') AS lne_digits,
    UPPER(TRIM(env)) AS dpi_environment,
    clean_date(TRY_TO_DATE(CAST(paypdt AS STRING),'yyyyMMdd')) AS payment_date,
    CAST(payamt AS DECIMAL(18,2)) AS payment_amount,
    UPPER(TRIM(paytyp)) AS payment_type
  FROM it_bronze_prod.dpi.blpymt60
  WHERE paypdt IS NOT NULL
),
pay_scoped AS (
  SELECT
    CASE WHEN LENGTH(exc_digits)=6 AND LENGTH(lne_digits) BETWEEN 1 AND 4
         THEN CONCAT(exc_digits, LPAD(lne_digits,4,'0')) END AS btn,
    dpi_environment, payment_date, payment_amount, payment_type
  FROM pay_raw
  WHERE payment_date IS NOT NULL
),
pay_agg_all AS (
  SELECT btn, dpi_environment,
         COUNT(*) AS payments_count_all_time,
         COALESCE(SUM(payment_amount),0) AS payments_sum_all_time
  FROM pay_scoped
  WHERE btn RLIKE '^[0-9]{10}$'
  GROUP BY 1,2
),
pay_last AS (
  SELECT btn, dpi_environment, payment_date AS last_payment_date,
         payment_type AS last_payment_type, payment_amount AS last_payment_amount
  FROM (
    SELECT *, ROW_NUMBER() OVER (
      PARTITION BY btn, dpi_environment
      ORDER BY payment_date DESC, payment_amount DESC
    ) rn
    FROM pay_scoped
    WHERE btn RLIKE '^[0-9]{10}$'
  )
  WHERE rn=1
),
pay_m AS (
  SELECT btn, dpi_environment,
         TO_DATE(DATE_TRUNC('month', payment_date)) AS month_start,
         SUM(payment_amount) AS month_payments,
         COUNT(*) AS month_payment_cnt
  FROM pay_scoped
  WHERE btn RLIKE '^[0-9]{10}$'
  GROUP BY 1,2,3
),
pay_roll AS (
  SELECT btn, dpi_environment,
         SORT_ARRAY(COLLECT_LIST(named_struct(
           'month_start', month_start, 'amount', month_payments, 'count', month_payment_cnt
         ))) AS last_12_mo_payments,
         AVG(month_payments) AS avg_monthly_payment_12m,
         SUM(month_payment_cnt) AS payments_count_12m
  FROM pay_m
  WHERE month_start >= ADD_MONTHS(DATE_TRUNC('month', current_date()), -11)
  GROUP BY 1,2
),

pay_types_12m_src AS (
  SELECT
    btn,
    dpi_environment,
    payment_date,
    payment_amount,
    payment_type,
    CASE payment_type
      WHEN '1AA' THEN 'One Time ACH - Fiserv'
      WHEN '1AC' THEN 'One Time Credit/Debit Card - Fiserv'
      WHEN '1CA' THEN 'Convenience Fee (ACH)'
      WHEN '1CC' THEN 'Convenience Fee (Card)'
      WHEN '1DP' THEN 'Deposit'
      WHEN '1EA' THEN 'One Time ACH - eComm'
      WHEN '1EC' THEN 'One Time Credit/Debit Card - eComm'
      WHEN '1FA' THEN 'One Time ACH - online'
      WHEN '1FC' THEN 'One Time Credit/Debit Card - online'
      WHEN '1IA' THEN 'One Time ACH - IVR'
      WHEN '1IC' THEN 'One Time Credit/Debit Card - IVR'
      WHEN '1MA' THEN 'One Time ACH - Mobile'
      WHEN '1MC' THEN 'One Time Credit/Debit Card - Mobile'
      WHEN '1PA' THEN 'One Time ACH - Fiserv'
      WHEN '1PC' THEN 'One Time Credit/Debit Card - Partner'
      WHEN '1WA' THEN 'One Time ACH - Walled Garden'
      WHEN '1XA' THEN 'One Time ACH - Express'
      WHEN '1XC' THEN 'One Time Credit/Debit Card - Express'
      WHEN '53B' THEN 'Paper Checks, Fifth Third Bank'
      WHEN 'APA' THEN 'Auto Pay ACH'
      WHEN 'APC' THEN 'Auto Pay Credit/Debit'
      WHEN 'B01' THEN 'Bank One Regulated'
      WHEN 'B02' THEN 'Bank One Regulated & Non-Regulated'
      WHEN 'B03' THEN 'Bank One Regulated & Non-Regulated'
      WHEN 'B05' THEN 'Bank One Regulated & Non-Regulated'
      WHEN 'B07' THEN 'CheckFreePay Regulated & Non-Regulated'
      WHEN 'B08' THEN 'Bank One Regulated'
      WHEN 'BD'  THEN 'Bank Draft / Auto Debit'
      WHEN 'CA1' THEN 'Electronic Check with Convenience Fee'
      WHEN 'CC1' THEN 'Credit Card with Convenience Fee'
      WHEN 'CCD' THEN 'Credit Card'
      WHEN 'CHF' THEN 'Checkfree payments'
      WHEN 'CKI' THEN 'Check/Money Order Payment'
      WHEN 'CKM' THEN 'Checks and Money Orders'
      WHEN 'COL' THEN 'Collection Agency Payment'
      WHEN 'COM' THEN 'Co-Mingled Payment / Check split'
      WHEN 'CP1' THEN 'Pinless Debit with Convenience Fee'
      WHEN 'CSH' THEN 'Cash'
      WHEN 'DEP' THEN 'Deposit'
      WHEN 'D01' THEN 'Pinless Credit Card Payment'
      WHEN 'D04' THEN 'Online Bill Pay - Debit Card'
      WHEN 'D05' THEN 'Debit Cards EBPP'
      WHEN 'EAS' THEN 'Eastern Account Services'
      WHEN 'FA1' THEN 'ACH / ECP Payment (FiServ/Bill Matrix)'
      WHEN 'FC1' THEN 'One-Time Credit Card Payment (FiServ/Bill Matrix)'
      WHEN 'FCS' THEN 'First Collection Services'
      WHEN 'FP1' THEN 'Pinless Credit Card Payment (FiServ/Bill Matrix)'
      WHEN 'GLB' THEN 'Globys credit card'
      WHEN 'KEN' THEN 'Collections agencies'
      WHEN 'MBW' THEN 'McCarthy, Burgess, & Wolff'
      WHEN 'MSC' THEN 'RPPS/Home Banking'
      WHEN 'O02' THEN 'Online Bill Pay-Bank Account Regulated & Non-Regulated'
      WHEN 'O04' THEN 'Online Bill Pay - Bank Account'
      WHEN 'O05' THEN 'E-checks EBPP'
      WHEN 'O07' THEN 'Online Bill Pay-Bank Account Regulated & Non-Regulated'
      WHEN 'O08' THEN 'Online Bill Pay-Bank Account Regulated'
      WHEN 'P02' THEN 'PCI Regulated & Non-Regulated'
      WHEN 'P03' THEN 'PCI Regulated & Non-Regulated'
      WHEN 'P04' THEN 'PCI - Credit Card'
      WHEN 'P08' THEN 'PCI Regulated'
      WHEN 'PAC' THEN 'Payment Agency Collection'
      WHEN 'PR1' THEN 'Home Banking (ACI, Princeton)'
      WHEN 'PR5' THEN 'Official Payments - Home Banking'
      WHEN 'R01' THEN 'Rochester Lock Box'
      WHEN 'R02' THEN 'Rochester Lock Box'
      WHEN 'R03' THEN 'Rochester Lock Box'
      WHEN 'R04' THEN 'JP Morgan Chase'
      WHEN 'R05' THEN 'Rochester Lockbox / JP Morgan Chase'
      WHEN 'R06' THEN 'JP Morgan Chase Regulated'
      WHEN 'R07' THEN 'JP Morgan Chase Regulated'
      WHEN 'R08' THEN 'PCI Regulated & Non-Regulated'
      WHEN 'RCP' THEN 'Research Correction via Payments'
      WHEN 'RUI' THEN 'RUI Credit Services'
      WHEN 'SET' THEN 'Settlement Payments'
      WHEN 'SPS' THEN 'Summary Spread / Parent/Child Spread'
      WHEN 'SRS' THEN 'Parent/Child Spread'
      WHEN 'STE' THEN 'Mailboxes Plus'
      WHEN 'TA1' THEN 'Electronic Checks through IVR/Salesforce / Paymentech'
      WHEN 'TC1' THEN 'One-time credit card or PIN-less debit (Frontier customer) / Paymentech'
      WHEN 'TC4' THEN 'WV Trickle Cash'
      WHEN 'TCA' THEN 'Verizon Trickle Cash'
      WHEN 'TCR' THEN 'Recurring Credit Card / Paymentech'
      WHEN 'TFT' THEN 'Trickle Cash FL/TX'
      WHEN 'TP1' THEN 'One Time Pinless Debit / Paymentech'
      WHEN 'TRC' THEN 'Recurring Credit Card Payment'
      WHEN 'USP' THEN 'Us Payments - Payment Kiosks (Tampa)'
      WHEN 'VIE' THEN 'Verizon Historic Info / Pre-Conversion Payments'
      WHEN 'VVE' THEN 'VZN Electronic Payment'
      WHEN 'VVR' THEN 'VZN Remittance Payment'
      WHEN 'WRO' THEN 'Written off Account'
      WHEN '711' THEN 'Non-Regulated Payment'
      ELSE 'UNKNOWN'
    END AS payment_type_description
  FROM pay_scoped
  WHERE btn RLIKE '^[0-9]{10}$'
    AND payment_date >= ADD_MONTHS(current_date(), -12)
),
pay_types_12m AS (
  SELECT
    btn,
    dpi_environment,
    COLLECT_LIST(
      named_struct(
        'payment_date', payment_date,
        'amount',       payment_amount,
        'type',         payment_type,
        'type_desc',    payment_type_description
      )
    ) AS payment_ledger_last_12m,
    COLLECT_SET(payment_type) AS payment_types_last_12m
  FROM pay_types_12m_src
  GROUP BY 1,2
)
SELECT
  b.btn,
  b.dpi_environment,
  COALESCE(pall.payments_count_all_time,0) AS payments_count_all_time,
  COALESCE(pall.payments_sum_all_time,0)   AS payments_sum_all_time,
  COALESCE(pr.avg_monthly_payment_12m,0)   AS avg_monthly_payment_12m,
  COALESCE(pr.payments_count_12m,0)        AS payments_count_12m,
  pl.last_payment_date,
  pl.last_payment_type,
  pl.last_payment_amount,
  COALESCE(pt.payment_types_last_12m,  ARRAY()) AS payment_types_last_12m,
  COALESCE(pr.last_12_mo_payments,     ARRAY()) AS last_12_mo_payments,
  COALESCE(pt.payment_ledger_last_12m, ARRAY()) AS payment_ledger_last_12m
FROM (
  SELECT DISTINCT btn, dpi_environment
  FROM pay_scoped
  WHERE btn RLIKE '^[0-9]{10}$'
) b
LEFT JOIN pay_agg_all   pall ON pall.btn = b.btn AND pall.dpi_environment = b.dpi_environment
LEFT JOIN pay_roll      pr   ON pr.btn   = b.btn AND pr.dpi_environment   = b.dpi_environment
LEFT JOIN pay_last      pl   ON pl.btn   = b.btn AND pl.dpi_environment   = b.dpi_environment
LEFT JOIN pay_types_12m pt   ON pt.btn   = b.btn AND pt.dpi_environment   = b.dpi_environment;


OPTIMIZE teamconnect.pca_prism.01_tbl_account_prism_payments
ZORDER BY (btn, dpi_environment);

